In [1]:
import pandas as pd
import requests, json
import numpy as np
import folium
import re
import json
import datetime
import xml.etree.ElementTree as Xml

In [27]:
hermap_filename = 'data_comparision/heremap_road_points.json'
sumo_filename = 'data_comparision/sumo_road_points.json'

with open(hermap_filename) as f:
    heremap_json = json.load(f)
    
with open(sumo_filename) as f:
    sumo_json = json.load(f)

In [28]:
correction_offset = [0.00001, -0.00007]  # self.get_correction_offset(sumo_road_points, heremap_road_points)

offsetted_sumo_road_points = sumo_json.copy()
for k in offsetted_sumo_road_points.values():
    for l in k:
        l[0] += correction_offset[0]
        l[1] += correction_offset[1]

In [29]:

osm=folium.Map(location=[12.96984,77.74992],zoom_start=20)

for road_name, list_points in offsetted_sumo_road_points.items():
    folium.PolyLine(locations=list_points,opacity=0.75,weight=4).add_to(osm)
    
for road_name, list_points in heremap_json.items():
    folium.PolyLine(locations=list_points,color = 'red', opacity=0.75,weight=4).add_to(osm)

In [30]:
        
osm